In [1]:
import pymysql.cursors
# Connect to the database
import numpy as np
import pandas as pd

#DB 불러오기

In [2]:
connection = pymysql.connect(host='localhost',
        user='practice',
        password='6725',
        db='tip',
        charset='utf8',
        cursorclass=pymysql.cursors.DictCursor)

In [3]:
try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM tip.tips;"
        cursor.execute(sql)
        result = cursor.fetchall()
        tips = pd.DataFrame(result) #불러온 db를 df로 저장
finally:
    connection.close()

In [4]:
tips #불러온 데이터셋 확인

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
271,20.65,3.35,Male,No,None,Dinner,5
272,16.99,1.01,Female,None,Sun,Dinner,2
273,10.34,1.66,None,No,Sun,Dinner,3
274,20.65,3.35,Male,No,None,Dinner,4


In [5]:
tip_rate = tips.tip / tips.total_bill * 100

In [6]:
tips['tip_rate'] = tip_rate.round()

In [7]:
tips.isnull().sum() #결측치 확인

total_bill     0
tip            0
sex            7
smoker         7
day           14
time           0
size           0
tip_rate       0
dtype: int64

인코딩(바이너리, 라벨)

In [8]:
tips["sex"] = tips["sex"].replace({"Female" : 0, "Male" : 1})
tips["smoker"] = tips["smoker"].replace({"No" : 0, "Yes" : 1})
tips["day"] = tips["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3})
tips["time"] = tips["time"].replace({"Lunch" : 0, "Dinner" : 1})

#결측치제거

In [9]:
from sklearn.impute import SimpleImputer # 결측치제거 모듈 호출

In [10]:
y = tips['tip'] #목표변수 지정

In [11]:
tips_array = tips.drop('tip', axis = 1) #데이터프레임에서 목표변수를 제거한 데이터셋 생성, axis는 적용할 축(0 = index, 1 = columns)

In [12]:
tips_array = tips_array.values #array 전환, imputer가 df를 읽지 못하기때문

In [13]:
# define imputer
imputer = SimpleImputer(strategy='mean')

# fit on the dataset
imputer.fit(tips_array)

SimpleImputer()

In [14]:
# transform the dataset
Xtrans = imputer.transform(tips_array)

# print total missing
print('Missing: %d' % sum(np.isnan(Xtrans).flatten()))

Missing: 0


In [15]:
df_tips = pd.DataFrame(Xtrans) #결측치 제거 데이터셋을 데이터 프레임으로 전환

In [16]:
df_tips.describe() #데이터프레임 확인

,0,1,2,3,4,5,6
count,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000
mean,19.478768,0.635688,0.345725,1.812977,0.753623,2.655797,15.684783
std,8.518097,0.475958,0.470387,1.131107,0.431683,1.005983,6.067971
min,3.070000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
25%,13.385000,0.000000,0.000000,1.000000,1.000000,2.000000,13.000000
50%,17.550000,1.000000,0.000000,2.000000,1.000000,2.000000,16.000000
75%,22.890000,1.000000,1.000000,3.000000,1.000000,3.000000,19.000000
max,50.810000,1.000000,1.000000,3.000000,1.000000,6.000000,71.000000


RFE

In [17]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel='linear')

In [18]:
# define RFE
rfe = RFE(estimator, n_features_to_select=4) #4개 변수 추출
# fit RFE
select = rfe.fit(df_tips, y) #y는 목표변수 tip
select.support_ #선택된 값 확인

array([ True, False,  True, False, False,  True,  True])

In [19]:
# summarize all features #결과값 표시 0,3,4,5 선택
for i in range(df_tips.shape[1]):
  print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected=True, Rank: 1
Column: 1, Selected=False, Rank: 4
Column: 2, Selected=True, Rank: 1
Column: 3, Selected=False, Rank: 3
Column: 4, Selected=False, Rank: 2
Column: 5, Selected=True, Rank: 1
Column: 6, Selected=True, Rank: 1


Regression feature selection

In [20]:
#feature selection 호출
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression 

In [21]:
# define feature selection 선택 종류와 개수 설정
fs = SelectKBest(score_func=f_regression, k=4)

In [22]:
#실행
# apply feature selection
X_selected = fs.fit_transform(df_tips, y)

print(X_selected.shape)
pd.DataFrame(X_selected)

(276, 4)


,0,1,2,3
0,16.99,0.000000,2.0,6.0
1,10.34,1.000000,3.0,16.0
2,21.01,1.000000,3.0,17.0
3,23.68,1.000000,2.0,14.0
4,24.59,0.000000,4.0,15.0
...,...,...,...,...
271,20.65,1.000000,5.0,16.0
272,16.99,0.000000,2.0,6.0
273,10.34,0.635688,3.0,16.0
274,20.65,1.000000,4.0,16.0


PCA

In [23]:
df_tips.columns = ['total_bill','sex','smoker','day','time','size','tip_rate']

In [24]:
df_tips_2 = df_tips[['sex','smoker','day','time','size','tip_rate','total_bill']]

In [25]:
df_tips

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0.000000,0.000000,3.000000,1.0,2.0,6.0
1,10.34,1.000000,0.000000,3.000000,1.0,3.0,16.0
2,21.01,1.000000,0.000000,3.000000,1.0,3.0,17.0
3,23.68,1.000000,0.000000,3.000000,1.0,2.0,14.0
4,24.59,0.000000,0.000000,3.000000,1.0,4.0,15.0
...,...,...,...,...,...,...,...
271,20.65,1.000000,0.000000,1.812977,1.0,5.0,16.0
272,16.99,0.000000,0.345725,3.000000,1.0,2.0,6.0
273,10.34,0.635688,0.000000,3.000000,1.0,3.0,16.0
274,20.65,1.000000,0.000000,1.812977,1.0,4.0,16.0


In [26]:
df_tips_2

,sex,smoker,day,time,size,tip_rate,total_bill
0,0.000000,0.000000,3.000000,1.0,2.0,6.0,16.99
1,1.000000,0.000000,3.000000,1.0,3.0,16.0,10.34
2,1.000000,0.000000,3.000000,1.0,3.0,17.0,21.01
3,1.000000,0.000000,3.000000,1.0,2.0,14.0,23.68
4,0.000000,0.000000,3.000000,1.0,4.0,15.0,24.59
...,...,...,...,...,...,...,...
271,1.000000,0.000000,1.812977,1.0,5.0,16.0,20.65
272,0.000000,0.345725,3.000000,1.0,2.0,6.0,16.99
273,0.635688,0.000000,3.000000,1.0,3.0,16.0,10.34
274,1.000000,0.000000,1.812977,1.0,4.0,16.0,20.65


In [28]:
#PCA호출
from sklearn.decomposition import PCA

In [29]:
# define the transform #PCA는 목표변수 설정이 필요하지 않음
trans = PCA(n_components=7)

# transform the data
X_dim = trans.fit_transform(df_tips)

# summarize data after the transform
pd.DataFrame(X_dim)

,0,1,2,3,4,5,6
0,-0.943793,-9.978000,-1.119794,-0.473670,0.655432,-0.209858,-0.096117
1,8.639167,-2.788559,-1.506182,0.817845,-0.279061,-0.112889,-0.132517
2,-1.029982,1.770453,-1.237027,0.154715,-0.181437,-0.336766,-0.138010
3,-4.500227,-0.180041,-1.039604,-0.930552,-0.208595,-0.590408,-0.132796
4,-5.120265,1.131814,-1.192704,0.868022,0.840977,0.011514,-0.111118
...,...,...,...,...,...,...,...
271,-1.128743,0.779437,-0.376074,2.268243,-0.220934,0.120805,0.219179
272,-0.945274,-9.975132,-1.104260,-0.546885,0.589985,0.120170,-0.122801
273,8.641848,-2.791964,-1.480652,0.795800,0.077306,-0.047036,-0.117267
274,-1.071454,0.743131,-0.254154,1.304118,-0.249176,-0.103010,0.229570


In [30]:
# define the transform #PCA는 목표변수 설정이 필요하지 않음
trans2 = PCA(n_components=7)

# transform the data
X_dim2 = trans.fit_transform(df_tips_2)

# summarize data after the transform
pd.DataFrame(X_dim2)

,0,1,2,3,4,5,6
0,-0.943793,-9.978000,-1.119794,-0.473670,0.655432,-0.209858,-0.096117
1,8.639167,-2.788559,-1.506182,0.817845,-0.279061,-0.112889,-0.132517
2,-1.029982,1.770453,-1.237027,0.154715,-0.181437,-0.336766,-0.138010
3,-4.500227,-0.180041,-1.039604,-0.930552,-0.208595,-0.590408,-0.132796
4,-5.120265,1.131814,-1.192704,0.868022,0.840977,0.011514,-0.111118
...,...,...,...,...,...,...,...
271,-1.128743,0.779437,-0.376074,2.268243,-0.220934,0.120805,0.219179
272,-0.945274,-9.975132,-1.104260,-0.546885,0.589985,0.120170,-0.122801
273,8.641848,-2.791964,-1.480652,0.795800,0.077306,-0.047036,-0.117267
274,-1.071454,0.743131,-0.254154,1.304118,-0.249176,-0.103010,0.229570


In [29]:
trans.explained_variance_.round()

array([78., 31.,  1.,  1.,  0.,  0.,  0.])

Nomalization, Standadization

In [25]:
#Nomalization 호출
from sklearn.preprocessing import MinMaxScaler
#Standardization 호출
from sklearn.preprocessing import StandardScaler


In [26]:
#Nomalization 적용
# define the scaler
trans = MinMaxScaler()

# transform the data
X_norm = trans.fit_transform(df_tips)

# summarize data after the transform
print(X_norm[:3, :])

[[0.29157939 0.         0.         1.         1.         0.2
  0.02985075]
 [0.1522832  1.         0.         1.         1.         0.4
  0.17910448]
 [0.3757855  1.         0.         1.         1.         0.4
  0.19402985]]


In [27]:
#Standardization 적용
sc = StandardScaler()
X_train = sc.fit_transform(df_tips)
df_std = pd.DataFrame(X_train)

In [28]:
#Nomalization 확인

In [29]:
pd.DataFrame(X_norm).describe()#값이 0과 1

,0,1,2,3,4,5,6
count,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000
mean,0.343711,0.635688,0.345725,0.604326,0.753623,0.331159,0.174400
std,0.178427,0.475958,0.470387,0.377036,0.431683,0.201197,0.090567
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.216066,0.000000,0.000000,0.333333,1.000000,0.200000,0.134328
50%,0.303310,1.000000,0.000000,0.666667,1.000000,0.200000,0.179104
75%,0.415165,1.000000,1.000000,1.000000,1.000000,0.400000,0.223881
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [30]:
#Standardization 확인

In [31]:
df_std.describe().round() #평균은 0 분산은 1

,0,1,2,3,4,5,6
count,276.0,276.0,276.0,276.0,276.0,276.0,276.0
mean,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0
min,-2.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0
25%,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-0.0
50%,-0.0,1.0,-1.0,0.0,1.0,-1.0,0.0
75%,0.0,1.0,1.0,1.0,1.0,0.0,1.0
max,4.0,1.0,1.0,1.0,1.0,3.0,9.0
